# Welcome to Data Cube Chile <img align="right" width="200" src="http://datacubechile.cl/wp-content/uploads/2022/06/datacube-chile-transparente.png">

This notebook will introduce new users to working with the Data Cube Chile and EASI Jupyter notebooks.

Data Cube Chile uses the [Open Data Cube](https://opendatacube.org) software and CSIRO's [Earth Analytics Science & Innovation](https://research.csiro.au/easi/) (EASI) platform).

Useful links & references:
- #### https://opendatacube.readthedocs.io/
- #### https://github.com/opendatacube
- ##### https://github.com/csiro-easi/easi-notebooks
- ##### https://knowledge.dea.ga.gov.au/dea-notebooks/
- ##### https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Beginners_guide
- ##### https://raw.githubusercontent.com/opendatacube/datacube-core/develop/docs/cheatsheets/ODC_Cheatsheet.pdf
<div>
    <a href="https://github.com/opendatacube/datacube-core/tree/develop/docs/cheatsheets">
        <img width="600" src="https://github.com/opendatacube/datacube-core/blob/develop/docs/cheatsheets/ODC_Cheatsheet.jpg?raw=true">
    </a>
</div>

## Notebook setup

A notebook consists of cells that contain either __text descriptions__ or __python code__ for performing operations on data.

Start by clicking on the cell below to select it. Then execute a selected cell (or each cell in sequence) by clicking the "play" button (in the toolbar above) or pressing `Shift`+`Enter`.

Each cell will show an asterisk icon <font color='#999'>[*]:</font> when it is running. Once this changes to a number, the cell has finished.

#### Imports
These are a standard set of imports that we use across many notebooks

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import rioxarray
import pandas as pd
from datetime import datetime
pd.set_option('max_colwidth', 120)

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from datacube.utils.rio import configure_s3_access

import matplotlib.pyplot as plt

# Python
import sys, os, re

In [ ]:
# Optional EASI tools
sys.path.append(os.path.expanduser('~/IALE-demo/scripts'))
from easi_tools import EasiDefaults
import notebook_utils
easi = EasiDefaults()

from app_utils import animated_timeseries
import app_utils

## Introduction to Dask

Firstly, a few notes on terminology. A Dask Cluster is comprised of a __client__, a __scheduler__, and __workers__. These terms will be used throughout this tutorial. Figure 1 below shows the relationship between each of these components. The __client__ submits tasks to the __scheduler__, which decides how to submit the tasks to individual workers. During this process, the scheduler creates what is called a __Task Graph__. This is essentially a map of the tasks that need to be carried out. Figure 2 shows an example of a simple task graph (see https://docs.dask.org/en/stable/graphs.html for more information. __Workers__ carry out the actual calculations and either store the results or send them back to the client.

<div>
    <span style="border:solid 1px #888;float:left;padding:10px;margin-right:25px;width:550px">
        <img src="images/distributed-overview.png">
        <figcaption><em>Figure 1. Overview of a Dask Cluster.</em></figcaption>
    </span>
</div>

In [ ]:
def increment(i):
    return i + 1

def add(a, b):
    return a + b

x = 1
y = increment(x)
z = add(y, 10)

In [ ]:
print(f'The value of z is {z}')

<div>
    <span style="border:solid 1px #888;float:left;padding:10px;margin-left:25px;width:150px">
        <img src="images/dask-simple.png">
        <figcaption><em>Figure 2. A simple Task Graph.</em></figcaption>
    </span>
</div>

Dask has several core data types, including __Dask DataFrames__ and __Dask Arrays__. Essentially, Dask DataFrames are parallelized Pandas DataFrames (Figure 3) and Dask Arrays are parallelized Numpy arrays (Figure 4).
<div>
    </span>
    <span style="border:solid 1px #888;float:left;padding:10px;margin-left:25px;width:300px">
        <img src="images/dask-array.svg">
        <figcaption><em>Figure 3. A Dask Array is a subset of the NumPy <code>ndarray</code> interface using blocked algorithms, cutting up the large array into many small arrays.</em></figcaption>
    </span>
</div>

More complex examples result in more complex Task Graphs:
<div>
    <img src="https://blog.dask.org/images/custom-etl.png">
</div>

We can also use the Dask Dashboard to watch our calculations progress in parallel:
<div>
    <img src="https://blog.dask.org/images/task-stream-custom-etl.gif">
</div>

## Dask computing environment

In EASI, each notebook starts by defining a Dask cluster for the notebook to use.

> For more information regarding Dask, see the addtional introductory notebooks on Dask or visit the [Dask Website](https://dask.org).

The are two main methods for setting up your dask cluster: 
1. **Local dask cluster**
    - Provides a dask multiprocessing environment on your Jupyter node. Useful for processing data volumes that don't exceed the Jupyter node limits, which are currently set at `cores = 8, memory = 32 GB` (2x large)


1. **Dask Gateway**
    - Provides a scalable compute cluster in EASI for your use. You can (*should*) use the same cluster across each of your notebooks (a separate cluster per notebook would unnessarily use EASI resources).
    - For most notebooks and data analysis start with `2 to 4 workers` (adaptive). Dask gateway is limited to 20 workers per user.
    - It is normal for this step to take **3 to 5 minutes** if new computing nodes need to be generated

**This notebook will use a local cluster**

### Local dask cluster

For local cluster options, see https://docs.dask.org/en/latest/setup/single-distributed.html

The Dask Dashboard link shown after the following cell is a helpful resource to explore the activity and state of your dask cluster.

In [ ]:
cluster, client = notebook_utils.initialize_dask(use_gateway=False)
display(cluster if cluster else client)

- simple load landsat 8 data example
- calculate NDVI filtered by landuse
- calculate temperature timeseries by landuse

## Access public requester pays buckets

EASI OpenDataCube can index and use datasets stored in public S3 "requester pays" buckets. Requester pays means that use of the data is charged at the time of use. The charges are relatively low for normal exploratory analysis and within the same Data Center.

> For larger analyses or between Data Centers please contact us for advice as there may be more cost-effective ways to do your analysis that we can explore with you.

To use data in public requester pays buckets, run the following code (once per dask cluster):

**All Landsat (e.g. landsat5_c2l2_sr, landsat9_c2l2_st, etc) and Sentinel-2 (s2_l2a) products require this setting**

In [ ]:
"""This function obtains credentials for S3 access and passes them on to
   processing threads, either local or on dask cluster.
   Note that AWS credentials may need to be renewed between sessions or
   after a period of time."""

from datacube.utils.aws import configure_s3_access
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

# If not using a dask cluster then remove 'client':
# configure_s3_access(aws_unsigned=False, requester_pays=True)

## Connect to the OpenDataCube
Your EASI Hub environment has been setup with default credentials to access the EASI OpenDataCube 

In [ ]:
import datacube
dc = datacube.Datacube()
datacube.__version__

### List available products

Get all available products and list them along with selected properties.

> View available products and data coverage at the data cube Explorer: https://explorer.datacubechile.cl

In [ ]:
products = dc.list_products()

# The output of list_products() changed between datacube versions 1.8.4 and 1.8.6
selected_columns = products.columns
if 'default_crs' not in selected_columns:
    selected_columns = ["name", "description", "platform", "crs", "resolution"]
products[selected_columns]

In [ ]:
measurements = dc.list_measurements()
measurements.loc['landsat8_c2l2_sr']

### Load some data
#### Set some query parameters

In [ ]:
study_area_lat = (-32.6, -32.45)
study_area_lon = (-71.5, -71.35)

product = 'landsat8_c2l2_sr'

set_time = ('2024-04-01', '2025-03-31')

set_crs = 'EPSG:32719'

set_resolution = (-30, 30) # (N, E) - El punto de referencia para la mayoría de los geotiffs está en la parte superior izquierda.

In [ ]:
app_utils.display_map(x=study_area_lon, y=study_area_lat)

#### Use `dc.load()` to load data as an <span style="font-size:22px">xarray Dataset</span><img style="float:right" width="150px" src="https://docs.xarray.dev/en/stable/_static/logos/Xarray_Logo_RGB_Final.png">

In [ ]:
data = dc.load(
    product=product, 
    latitude=study_area_lat,
    longitude=study_area_lon,
    time=set_time,
    output_crs=set_crs,
    resolution=set_resolution,
    group_by='solar_day',
    dask_chunks={'time':1}
)

display(data)

### NOTE:
This next step is not normally done at this point and it is only being used for this workshop, as it forces Dask to load all of the data into memory. This is only possible here because we are loading a small amount of data.

This is not a very efficient way to work, but is useful to help in this workshop to make the visualisations run faster

To do more complex calculations, <span style="color:red;">do not use this step</span>

In [ ]:
data = data.compute()

In [ ]:
data

#### Visualise one band for one scene

In [ ]:
data.red.isel(time=3).plot(robust=True, size=8, aspect=1)

#### Combine bands to load a true colour image

In [ ]:
data[['red','green','blue']].isel(time=3).to_array().plot.imshow(robust=True, size=8, aspect=1)

In [ ]:
data[['swir22','nir08','red']].isel(time=3).to_array().plot.imshow(robust=True, size=8, aspect=1)

#### Visualise all dates (true colour)

In [ ]:
data[['red','green','blue']].sel(time='2025').to_array().plot.imshow(col='time', col_wrap=4, robust=True, aspect=1)

#### All the clouds make it difficult to see the data, so we can change the scaling to show the land

In [ ]:
data[['red','green','blue']].sel(time='2025').to_array().plot.imshow(col='time', col_wrap=4, vmin=6000, vmax=12000, aspect=1)

### Cloud filtering

There are different ways to filter for clouds and each satellite has its own metadata flags. This example works with Landsat data

#### See https://explorer.datacubechile.cl/products/landsat8_c2l2_sr to look at flag definitions 

In [ ]:
measurements = dc.list_measurements().loc['landsat8_c2l2_sr']

# Separate lists of measurement data names and flag names
data_names = measurements[pd.isnull(measurements.flags_definition)].index
flag_names = measurements[pd.notnull(measurements.flags_definition)]

# Select one for use below
flag_names

In [ ]:
# Pandas table. First flags_definition measurement found
masking.describe_variable_flags(data.qa_pixel)

In [ ]:
data.qa_pixel.flags_definition

In [ ]:
flag_name = 'qa_pixel'

### Make a cloud mask

The `make_mask()` returns a mask where `True` corresponds to the selected bits and values. These may considered as _good_ or _bad_ pixel flag selections depending on the application and the `flag_definition`.

Define a dictionary of ___good___ pixel flags using values shown in the variable flags above `{'flag': 'value'}`.

>__NOTE:__ The examples below are designed to work with the Landsat flags. Other products will have different flag definitions.

In [ ]:
good_pixel_flags = {
    'nodata': False,
    'cloud': 'not_high_confidence',
    'cloud_shadow': 'not_high_confidence',
    'cirrus': 'not_high_confidence',
    'water': 'land_or_cloud'
}

Make a mask corresponding to the `good_pixel_flags` and plot the result.

Below, we use `**good_pixel_flags` in the function. The use of `**` with a python dictionary like this expands the dictionary into individual parameters, so the two lines below are identical:

```python
mask = masking.make_mask(good_data, **good_pixel_flags)

mask = masking.make_mask(good_data, nodata=False, cloud='not_high_confidence', cloud_shadow='not_high_confidence', cirrus='not_high_confidence', water='land_or_cloud')
```

In [ ]:
# Make the mask:
mask = masking.make_mask(data[flag_name], **good_pixel_flags)

# Apply the mask:
good_data = data.where(mask)

fig = good_data[['red','green','blue']].sel(time='2025').to_array().plot.imshow(col='time', col_wrap=4, vmin=6000, vmax=12000, aspect=1)

# Add background colour to plots
for ax in fig.axs.flat:
    ax.set_facecolor('#E8F4E2');

We can also summarise the percentage of good pixels per date

In [ ]:
pixels = mask.shape[1] * mask.shape[2]
percent = mask.sum(dim=['y','x']) / pixels *100
fig = percent.plot()
fig[0].axes.set_title('Percent good pixels by date');
fig[0].axes.set_xlabel('Date');
fig[0].axes.set_ylabel('%');

In [ ]:
good_data.where(percent.compute() > 40, drop=True).red.plot(col='time',col_wrap=4,robust=True)

In [ ]:
good_data = good_data.drop_sel(time=good_data.sel(time='2024-06-29').time)
good_data = good_data.where(percent.compute()>40, drop=True)
good_data.red.plot(col='time',col_wrap=4,robust=True)

### Convert values to Surface Reflectance

To save storage space, satellite data is stored as __Digital Numbers (DN)__, which are integers rather than floating point (decimal) values. For Landsat data, this results in valid integers between 7273 and 43636. These values can be used individually, although if combining bands or running calculations on data, these <span style="color:red;font-weight:bold;">MUST</span> be converted to __Surface Reflectance (SR)__ values first. 

### $SR = (DN * 0.0000275) - 0.2$

> Each satellite product (not just Landsat) has its own scaling factors. These should be investigated when using satellite data. Some are much easier, for example Sentinel 2 has a scaling factor of 1/10000 and an offset of 0. The offset of 0 means that the relationship between bands is not affected after scaling, so it is less critical, but still important to convert to Surface Reflectance before using the data.

For more information on Landsat, see https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products

<table style="float:left;clear:both;">
	<thead>
		<tr>
			<th>Science Product</th>
			<th>Scale Factor</th>
			<th>Additive<br>
			Offset</th>
			<th>Fill Value</th>
			<th>Data Type</th>
			<th>Valid Range</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td><a href="https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-reflectance">Surface Reflectance</a></td>
			<td>0.0000275</td>
			<td>-0.2</td>
			<td>0</td>
			<td>Unsigned 16-bit integer</td>
			<td>7273-43636&nbsp;</td>
		</tr>
		<tr>
			<td><a href="https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-temperature">Surface Temperature</a></td>
			<td>0.00341802</td>
			<td>149</td>
			<td>0</td>
			<td>Unsigned 16-bit integer</td>
			<td>293 - 65535&nbsp;</td>
		</tr>
	</tbody>
</table><div style="clear:both">&nbsp;</div>


To apply the equation, we can either just apply the calculation directly on the DataSet, or we can use the `to_f32` function from the OpenDataCube's Algorithm (`odc.algo`) library:

```python
to_f32(x, scale, offset)
```

In [ ]:
from odc.algo import to_f32

In [ ]:
# This should only be applied to the data bands. In this example, we select these by using the data_names list that we definied earlier
data_scaled = to_f32(good_data[data_names],0.0000275,-0.2)

In [ ]:
# The original data is in uint16 - unsigned 16-bit integers
print(f'Min: {data.red.min().values}, Max: {data.red.max().values}')
display(data)

In [ ]:
print(f'Min: {round(data_scaled.red.min().values.item(),3)}, Max: {round(data_scaled.red.max().values.item(),3)}')
display(data_scaled)

In [ ]:
data_scaled.red.isel(time=2).plot(robust=True, size=8, aspect=1, cmap='viridis')

### Calculate an index - NDVI

__Normalised Difference Vegetation Index__
<div style="float:left">
$$
NDVI = {{NIR - RED} \over {NIR + RED}}
$$
</div>

See https://gisgeography.com/ndvi-normalized-difference-vegetation-index/ for more information on NDVI

In [ ]:
data_scaled['ndvi'] = (data_scaled.nir08 - data_scaled.red)/(data_scaled.nir08 + data_scaled.red)
data_scaled

In [ ]:
data_scaled.ndvi.plot(vmin=0.25, vmax=0.75, cmap='summer_r', col='time', col_wrap=4, size=4, aspect=1)

In [ ]:
fig = data_scaled.ndvi.mean(dim=['x','y']).plot(figsize=(10,4))
fig[0].axes.set_title('Monthly mean NDVI');
fig[0].axes.set_xlabel('Date');
fig[0].axes.set_ylabel('NDVI');

In [ ]:
data_scaled.ndvi.median(dim='time').plot(robust=True,cmap='summer_r',size=6,aspect=1)

In [ ]:
# Export to GeoTIFF
data_scaled.ndvi.mean(dim='time').rio.to_raster("ndvi_example.tif")

In [ ]:
# Export to CSV
data_scaled.ndvi.mean(dim=['x','y']).to_dataframe().to_csv('ndvi_example.csv')

### EXTRA... combining products and advanced analysis

#### Load land cover data using the `like` parameter
https://www.gep.uchile.cl/Landcover_CHILE.html

In [ ]:
landcover = dc.load(
    product='landcover_chile_2014',
    time='2014',
    like=data)
landcover

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch

In [ ]:
nivel_1 = landcover.nivel_1.squeeze()

# Extract flags_definition
flags_definition = nivel_1.attrs.get("flags_definition", {}).get("data", {})
category_values = {int(key): name for key, name in flags_definition.get("values", {}).items()}

# Define colors for each category
# Ensure colors cover all categories in the dataset
landcover_colors = {
    0: "#FFFFFF",  # no data
    100: "#DA984A",  # Cultivos
    200: "#4B7A4E",  # Bosques
    300: "#8FAE61",  # Pastizales
    400: "#DBC369",  # Matorrales
    500: "#7C88C2",  # Humedales
    600: "#579BD8",  # Cuerpos de Agua
    700: "#FFFF",    # Empty
    800: "#B33826",  # Superficies Impermeables
    900: "#A39B90",  # Tierras desnudas
    1000: "#AFA1DC",  # Hielo y Nieves
    1100: "#FFFF",    # Empty
    1200: "#EBEBEB"  # Nubes
}

# Map colors to categories in flags_definition
colors = [landcover_colors[cat] for cat in sorted(category_values.keys())]

# Create colormap and normalization
cmap = ListedColormap(colors)
bounds = sorted(category_values.keys()) + [max(category_values.keys()) + 100]
norm = BoundaryNorm(bounds, cmap.N)

# Create legend elements
legend_elements = [
    Patch(facecolor=landcover_colors[cat], edgecolor="black", label=category_values[cat])
    for cat in sorted(category_values.keys())
]

# Manually create the figure and axes
fig, ax = plt.subplots(figsize=(8, 8))

# Plot using xarray's built-in plot
im = nivel_1.plot(
    cmap=cmap,
    norm=norm,
    ax=ax,
    add_colorbar=False
)

# Ensure the aspect ratio is correct
ax.set_aspect('equal')  # Set the aspect ratio to 'equal' to reflect true dimensions

# Add custom legend outside the main plot
ax.legend(
    handles=legend_elements,
    loc="upper left",
    bbox_to_anchor=(1.05, 1),  # Position legend outside the plot
    title="Categories"
)

# Add labels and title
ax.set_title("Land Cover Map - Nivel 1")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# Adjust layout to ensure space for the legend
plt.tight_layout()
plt.show()

In [ ]:
# Prepare to combine datasets
ndvi = data_scaled.ndvi
landcover_nivel_1 = landcover.nivel_1.squeeze() # This removes the time dimension (which only has one date and is not needed)
ndvi = ndvi.fillna(-9999)  # Replace NaN in ndvi with a real value, e.g., -9999 to avoid errors

# Create a version of landcover with the same dates as the ndvi data
landcover_broadcasted = landcover_nivel_1.broadcast_like(ndvi)

# Add the new multi-date landcover to the original data
data_scaled['landcover']=landcover_broadcasted
data_scaled

In [ ]:
ndvi_masked = data_scaled.ndvi.where(landcover_broadcasted != 0, drop=False)
ndvi_mean_by_landcover = ndvi_masked.groupby(data_scaled.landcover).mean(dim=["x", "y"], fill_value=0)

unique_landcover_values = sorted(set(ndvi_masked.groupby(data_scaled.landcover).groups.keys()))

# Map the numeric landcover values to descriptive names
landcover_categories = {
    0: "No data",
    100: "Cultivos",
    200: "Bosques",
    300: "Pastizales",
    400: "Matorrales",
    500: "Humedales",
    600: "Cuerpos de Agua",
    800: "Superficies Impermeables",
    900: "Tierras desnudas",
    1000: "Hielo y Nieves",
    1200: "Nubes",
}

# Assign the descriptive names as a new coordinate
ndvi_mean_by_landcover = ndvi_mean_by_landcover.assign_coords(
    landcover_category=("landcover", [landcover_categories.get(int(key), "Unknown") for key in unique_landcover_values])
)
ndvi_mean_by_landcover = ndvi_mean_by_landcover.where(
    ndvi_mean_by_landcover.landcover > 0, drop=True
).where(
    ndvi_mean_by_landcover.landcover <= unique_landcover_values.index(500), drop=True
)

In [ ]:
ndvi_mean_by_landcover

In [ ]:
import matplotlib.pyplot as plt

# Extract the data for plotting
time = ndvi_mean_by_landcover['time'].values
landcover_values = ndvi_mean_by_landcover['landcover_category'].values
ndvi_values = ndvi_mean_by_landcover.values

# Create the plot
plt.figure(figsize=(12, 6))

# Loop through each landcover category and plot its NDVI with the corresponding color
for category in landcover_values:
    landcover_val = list(landcover_categories.values()).index(category)
    landcover_code = next((key for key, value in landcover_categories.items() if value == category), None)
    color = landcover_colors.get(landcover_code, 'black')  # Default to black if category is missing
    plt.plot(
        ndvi_mean_by_landcover['time'],  # x-axis
        ndvi_mean_by_landcover.sel(landcover=landcover_val),  # y-axis
        label=category,
        color=color,
        linewidth=4
    )

# Add labels, legend, and title
plt.xlabel('Time')
plt.ylabel('NDVI')
plt.title('NDVI Over Time by Landcover Category')
plt.legend(title='Landcover Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ndvi_mean_by_landcover.to_dataframe().to_csv('ndvi_by_landuse.csv')